Page_No: 62 - 
# Top N List in each Partition - Nonunique Keys

In [1]:
topNnonUni = "./DataSet/topNnonUni.csv"
f1RDD = sc.textFile(topNnonUni)

#### Creating PairRDD

In [2]:
def parse(data):
    dataList = data.split(',')
    return (dataList[0], int(dataList[1]))
f1PRDD = f1RDD.map(parse)

In [3]:
# Broadcast the N value
nValue = sc.broadcast(3)

#### Removing Nonunique keys

In [4]:
f1PRDD = f1PRDD.reduceByKey(lambda x, y: x + y)

#### Partitioning the Pair RDD
* What is the right number of partitions? 
    * There is no magic bullet for calculating the number of partitions. 
    * It depends on the number of cluster nodes, the number of cores per server, and the size of RAM available. 
    * The general rule of thumb is to use (2 × num_executors × cores_per_executor) per executor.
        Here executors means slave 

In [5]:
def partitionSort(data):
    dataList = list(data)
    dataList = sorted(dataList, key=lambda x: x[1], reverse=True)
    if len(dataList) > nValue.value:
        dataList = [dataList[i] for i in range(0, 3)]
    return (dataList)

In [8]:
f2PRDD = f1PRDD.mapPartitions(partitionSort)

In [11]:
f2PRDD.glom().collect()

[[('C', 7)], [('G', 6), ('A', 5), ('B', 5)]]

In [13]:
f2PRDD.takeOrdered(nValue.value, key=lambda iter1: -iter1[1])

[('C', 7), ('G', 6), ('A', 5)]